This .ipynb contains preprocessing codes for GEFS reforecast
and GEFS reanalysis datasets


# Importing Libraries

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs  # for plotting map
from sklearn.preprocessing import MinMaxScaler

c:\Users\bobby\anaconda3\envs\research\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Reanalysis and Reforecast 16 x 19 --> 8 x 11

In [2]:
# reading variable files
PATH = 'C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\raw_preprocessing\\GEFS\\'
ds_reanalysis_tp = xr.open_dataset(PATH + 'GEFSv12-Reanalysis_tp_2000_2019.nc') # only 1 single reanalysis tp
ds_reforecast_apcp = xr.open_dataset(PATH + 'GEFSv12-Reforecast_apcp_2000_2019.nc') # may add more variables in future

# slicing dimensions 8 x 11 grid points
ds_reanalysis_tp = ds_reanalysis_tp.sel(lon=slice('102.5', '105.00'), lat=('2.5','2.25','2','1.75','1.5','1.25','1','0.75'))
ds_reforecast_apcp = ds_reforecast_apcp.sel(longitude=slice('102.5', '105.00'), latitude=('2.5','2.25','2','1.75','1.5','1.25','1','0.75'))

# starting from 2000-01-01 06:00:00 to 2019-12-31 18:00:00, total 29219 time steps
ds_reanalysis_tp = ds_reanalysis_tp.isel(time=slice(1, None)) 
del ds_reanalysis_tp.attrs['history'] # remove long history text

In [3]:
# ds_reanalysis_tp.isel(time=0).tp # checking magnitudes

# plotting check between reforecast and reanalysis per timestep

In [4]:
# timestep = 12
# p1m = ds_reanalysis_tp.isel(time=timestep).tp
# p2m = ds_reforecast_apcp.isel(time=timestep).tp

# fig = plt.figure(figsize=(7, 3.5))
# p = p1m.plot(x="lon", y="lat", 
#              subplot_kws={"projection": ccrs.PlateCarree()}, transform=ccrs.PlateCarree(), cmap="Blues")
# p.axes.coastlines()
# p.axes.gridlines(draw_labels=True)

In [5]:
# fig = plt.figure(figsize=(7, 3.5))
# p = p2m.plot(x="longitude", y="latitude", 
#              subplot_kws={"projection": ccrs.PlateCarree()}, transform=ccrs.PlateCarree(), cmap="Blues")
# p.axes.coastlines()
# p.axes.gridlines(draw_labels=True)

# split into train val test

In [6]:
ds_train_apcp = ds_reforecast_apcp.sel(time=slice('2000-01-01T06', '2014-01-01T06'))
ds_val_apcp = ds_reforecast_apcp.sel(time=slice('2014-01-01T12', '2016-12-31T12'))
ds_test_apcp = ds_reforecast_apcp.sel(time=slice('2016-12-31T18', '2019-12-31T18'))


y_train = ds_reanalysis_tp.sel(time=slice('2000-01-01T06', '2014-01-01T06'))
y_val = ds_reanalysis_tp.sel(time=slice('2014-01-01T12', '2016-12-31T12'))
y_test = ds_reanalysis_tp.sel(time=slice('2016-12-31T18', '2019-12-31T18'))

In [7]:
# time check for apcp

# reforecast_time_check = ds_reforecast_apcp.time.values.astype('datetime64[h]') # 29215
# print(reforecast_time_check)
# print(len(reforecast_time_check))

# official_time_check = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2020-01-01"), np.timedelta64(6, "h")) # 29219
# print(official_time_check)
# print(len(official_time_check))

# print('\nmissing time steps:')
# set(official_time_check) - set(reforecast_time_check) # missing data on 2012-05-17

def timecheck(train, val, test):
    '''
    Return missing time step in train, val and test split
    '''
    train_timecheck = np.arange(np.datetime64("2000-01-01T06"), np.datetime64("2014-01-01T12"), np.timedelta64(6, "h"))
    val_timecheck = np.arange(np.datetime64("2014-01-01T12"), np.datetime64("2016-12-31T18"), np.timedelta64(6, "h"))
    test_timecheck = np.arange(np.datetime64("2016-12-31T18"), np.datetime64("2020-01-01T00"), np.timedelta64(6, "h"))
    print('train:', set(train_timecheck) - set(train.time.values.astype('datetime64[h]')))
    print('val:', set(val_timecheck) - set(val.time.values.astype('datetime64[h]')))
    print('test:', set(test_timecheck) - set(test.time.values.astype('datetime64[h]')))
    return

timecheck(ds_train_apcp, ds_val_apcp, ds_test_apcp)
# timecheck(y_train, y_val, y_test) # reanalysis are all ok

train: {numpy.datetime64('2012-05-18T00','h'), numpy.datetime64('2012-05-17T18','h'), numpy.datetime64('2012-05-17T12','h'), numpy.datetime64('2012-05-17T06','h')}
val: set()
test: set()


In [8]:
ds_train_apcp = ds_train_apcp.resample(time='6H').asfreq()
ds_train_apcp.tp.values = ds_train_apcp.tp.interpolate_na(dim='time')
print(not ds_reforecast_apcp.tp.isnull().any()) # True if contains all numeric values

True


# Log transform and normalization

In [9]:
# log transform and min_max normalization for reforecast_apcp TRAINING DATASET
scaler_train_apcp = MinMaxScaler()
X_one_col = ds_train_apcp.tp.values.reshape([ds_train_apcp.tp.values.shape[0]*ds_train_apcp.tp.values.shape[1]*ds_train_apcp.tp.values.shape[2], 1])
X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
X_one_col_res = scaler_train_apcp.fit_transform(X_one_col) # scaler_train_apcp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train_apcp.inverse_transform(X_one_col_res) -1 only
ds_train_apcp.tp.values = X_one_col_res.reshape(ds_train_apcp.tp.values.shape)

# log transform and min_max normalization for reanalysis_tp TRAINING DATASET
scaler_train_tp = MinMaxScaler() 
y_one_col = y_train.tp.values.reshape([y_train.tp.values.shape[0]*y_train.tp.values.shape[1]*y_train.tp.values.shape[2], 1])
y_one_col = np.log10(y_one_col+1) # 10**X_one_col - 1 to scale back
y_one_col_res = scaler_train_tp.fit_transform(y_one_col) # scaler_train_apcp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train_apcp.inverse_transform(X_one_col_res) -1 only
y_train.tp.values = y_one_col_res.reshape(y_train.tp.values.shape)

In [10]:
def transform_val_test(val_test, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values
        
    else:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) 
        X_one_col_res = scaler_train.transform(X_one_col) 
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test.values

# reforecast
ds_val_apcp.tp.values = transform_val_test(ds_val_apcp.tp, scaler_train_apcp, True) # transforming val based on train
ds_test_apcp.tp.values = transform_val_test(ds_test_apcp.tp, scaler_train_apcp, True) # transforming val based on train

# reanalysis
y_val.tp.values = transform_val_test(y_val.tp, scaler_train_tp, True)
y_test.tp.values = transform_val_test(y_test.tp, scaler_train_tp, True)

def inverse_val_test(transformed_vt, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train_apcp, True/False
    Output: Inversed of transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec == True:
        X_one_col = transformed_vt.values.reshape([transformed_vt.values.shape[0]*transformed_vt.values.shape[1]*transformed_vt.values.shape[2], 1])
        X_one_col_res = 10**scaler_train.inverse_transform(X_one_col) -1
        transformed_vt.values = X_one_col_res.reshape(transformed_vt.values.shape)
        return transformed_vt.values
    
    else:
        X_one_col = transformed_vt.values.reshape([transformed_vt.values.shape[0]*transformed_vt.values.shape[1]*transformed_vt.values.shape[2], 1])
        X_one_col_res = scaler_train.inverse_transform(X_one_col)
        transformed_vt.values = X_one_col_res.reshape(transformed_vt.values.shape)
        return transformed_vt.values

# retrieving back original precipitation 
# do not use this yet
# ds_val_apcp.tp.values = inverse_val_test(ds_val_apcp.tp, scaler_train_apcp, True) 

# Checking if coordinates match the values

In [11]:
# ds_train_apcp.tp[0]

In [12]:
# ds_train_apcp.tp[0][0]

In [13]:
# ds_train_apcp.sel(latitude=2.25, longitude=103).isel(time=0)

# Converting to npy

In [14]:
# training
ds_train_apcp = ds_train_apcp.tp.values
ds_train_apcp = ds_train_apcp[..., np.newaxis]
print(ds_train_apcp.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\X_train_apcp.npy', ds_train_apcp)

# val
ds_val_apcp = ds_val_apcp.tp.values
ds_val_apcp = ds_val_apcp[..., np.newaxis]
print(ds_val_apcp.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\X_val_apcp.npy', ds_val_apcp)

# testing
ds_test_apcp = ds_test_apcp.tp.values
ds_test_apcp = ds_test_apcp[..., np.newaxis]
print(ds_test_apcp.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\X_test_apcp.npy', ds_test_apcp)

(20457, 8, 11, 1)
(4381, 8, 11, 1)
(4381, 8, 11, 1)


In [15]:
# training FOR REANALYSIS 
y_train = y_train.tp.values
y_train = y_train[..., np.newaxis]
print(y_train.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\y_train_tp.npy', y_train)

# val
y_val = y_val.tp.values
y_val = y_val[..., np.newaxis]
print(y_val.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\y_val_tp.npy', y_val)

# testing
y_test = y_test.tp.values
y_test = y_test[..., np.newaxis]
print(y_test.shape)
np.save('C:\\Users\\bobby\\Desktop\\.vscode\\1 UROP Research\\UROP v2\\data\\y_test_tp.npy', y_test)

(20457, 8, 11, 1)
(4381, 8, 11, 1)
(4381, 8, 11, 1)


# Others

In [16]:
# log transform and min_max normalization for reanalysis, MIGHT BE USED FOR CLASSIFICATION
# scaler_tp = MinMaxScaler()
# X_one_col = ds_reanalysis_tp.tp.values.reshape([ds_reanalysis_tp.tp.values.shape[0]*ds_reanalysis_tp.tp.values.shape[1]*ds_reanalysis_tp.tp.values.shape[2], 1])
# X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
# X_one_col_res = scaler_tp.fit_transform(X_one_col) # scaler_tp.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_tp.inverse_transform(X_one_col_res) -1 only
# ds_reanalysis_tp.tp.values = X_one_col_res.reshape(ds_reanalysis_tp.tp.values.shape)

In [17]:
# ds_reanalysis_tp

In [18]:
# Checks for REFORECAST
# ds_reforecast_apcp = xr.open_dataset('GEFSv12-Reforecast_apcp_2000_2019.nc')
# ds_reforecast_apcp = ds_reforecast_apcp.to_dataframe()['tp']
# ds_reforecast_apcp = ds_reforecast_apcp.loc[~ds_reforecast_apcp.index.duplicated(),:].unstack(level=[0,1])
# ds_reforecast_apcp = ds_reforecast_apcp.resample('D').mean()
# selected_index = ds_reforecast_apcp.index[np.where(ds_reforecast_apcp.isnull())[0]]
# missing_index = selected_index[np.where(selected_index.duplicated()==False)[0]]

# Checks for REANALYSIS
# ds_reanalysis_tp = xr.open_dataset('GEFSv12-Reanalysis_tp_2000_2019.nc')
# ds_reanalysis_tp = ds_reanalysis_tp.to_dataframe()['tp']
# ds_reanalysis_tp = ds_reanalysis_tp.loc[~ds_reanalysis_tp.index.duplicated(),:].unstack(level=[2,1])
# ds_reanalysis_tp = ds_reanalysis_tp.resample('D').mean()
# selected_index = ds_reanalysis_tp.index[np.where(ds_reanalysis_tp.isnull())[0]]
# missing_index = selected_index[np.where(selected_index.duplicated()==False)[0]]

# Plotting check

In [19]:
# Looking at the magnitude and general pattern of both reforecast and reanalysis 
# 1 YEAR SAMPLE 
# lat = 2
# lon = 101.5
# ds_reanalysis_tp.isel(time=slice(0,1468)).groupby(ds_reanalysis_tp.time[0:1468].dt.month).mean().tp.sel(lat=lat, lon=lon).plot()
# ds_reforecast_apcp.groupby(ds_reforecast_apcp.time.dt.month).mean().tp.sel(latitude=lat, longitude=lon).plot();

# FULL 20 YEARS
# from turtle import color

# lat = 3
# lon = 104
# ds_reanalysis_tp.groupby(ds_reanalysis_tp.time.dt.month).mean().tp.sel(lat=lat, lon=lon).plot(color='red', alpha=0.5)
# ds_reforecast_apcp.groupby(ds_reforecast_apcp.time.dt.month).mean().tp.sel(latitude=lat, longitude=lon).plot(color='blue', alpha=0.5);

In [20]:
# y = ds_reanalysis_tp.tp.values.astype(float) # 29220, 16, 19
# y = y[~np.isnan(y)]
# np.quantile(y, 0.95)

In [21]:
# print(not ds_reforecast_apcp.tp.isnull().any()) # True if contains all numeric values
# print(not ds_reanalysis_tp.tp.isnull().any()) # True if contains all numeric values

# Reanalysis 2 x 3 grid

In [22]:
# ds_target_6 = ds_target.sel(lat= [1.25, 1.5], lon= [103.5, 103.75, 104])
# # ds_target_6 = ds_target.sel(lat= [1.25, 1.5], lon= [103.75, 104])
# # ds_prec_6

# ds_target_6.isel(time=3).tp.plot() # 2 x 3

# # ds_target_6.tp[1].sel(lat=1.25, lon=103.75) # bottom left
# ds_target_6.tp[1].sel(lat=1.5, lon=103.75) # top left

# ds_target_6.tp[1] # as one can observe, 9.1 doesnt correspond to the array properly, where it is shown at the bottom row instead of the 1st row

# Reversing the lat coords

In [23]:
# ds_target_6 = ds_target_6.isel(lat=slice(None,None,-1))

In [24]:
# ds_target_6.tp[1].sel(lat=1.25, lon=103.75) # top left

In [25]:
# ds_target_6.tp[1]

# Old reanalysis nc file

In [26]:
# ds_sample = xr.open_dataset('combined-reanalysis.nc')

In [27]:
# ds_sample.tp.values[1]

In [28]:
# ds_sample.isel(time=3).tp.plot() # 2 x 2

In [29]:
# ds_sample.tp

# Exporting into .npy files

In [30]:
# data = ds_target.tp.values
# data = data[..., np.newaxis]
# # data.reshape(7305,2,2,1)
# # np.save('data.npy', data)
# data.shape

# Combining 3D arrays into 1 single 4D array

In [31]:
# x = np.zeros((1000, 90, 135)) # 1000 samples, 90 lat, 135 lon
# x1 = x[..., np.newaxis]
# print(x1.shape) # for 1 variable

# y = np.stack((x, x, x), axis = -1) # stacking 3 variables into 1 single 4D array
# print(y.shape)
# # y2 = np.stack((x, x, x))
# # y2.shape
# # y.shape
# # np.concatenate(x, axis=0)